# Week 1 Assignment 

## overview 

I am very interested in the gapminder dataset, this is a great resource for investigating for better insight about the factors in various countries all over the world in term of life quality, policy etc..

## about the data 

The data for this analysis comes from a subset of the GapMider project data. In this section I will examine the variables I am interested in more detail.

## Life Expectancy

according to wikipedia, Life expectancy is a statistical measure of the average time an organism is expected to live, based on the year of their birth, their current age and other demographic factors including sex. The life expectancy could be used to make inference for some aspects of the society

## Democracy Score

The democracy score comes from the Polity IV project. It is a summary measure of a country's democratic and free nature. -10 is the lowest value, 10 the highest. The GapMinder data set that I will analyzing is the polity score for 2009. To get a feel for this data take a look at the following figure provided by the Polity IV project authors:

<img src="democracy-index-2012.jpg">

## Democracy vs Life Expentacy Correlation 

while the relation between democracy and prosperity is well studied, not many paper talked about the directed relation beetween democracy and like expentacy. My interesting question is 

## Processing data 

I begin by importing the libraries needed for the analysis:

In [1]:
#import the necessary library 
import pandas as pd 
import numpy as np

Now I have python parse the csv file and print out some basic statistics about the data frame (df):

In [5]:
df = pd.read_csv("gapminder.csv",low_memory= False)
#print some basic features 
rows = len(df)
cols = len(df.columns)
print "the data has " + str(rows) + " rows"
print "the data has " + str(cols) + " columns"

the data has 213 rows
the data has 16 columns


here are 213 observations with 16 variables in the data frame. I need to clean up the raw data prior to the analysis. I will first change the varaible types for the variables of interst:

In [7]:
# Change the data type for variables of interest
df['polityscore'] = pd.to_numeric(df['polityscore'])
df['lifeexpectancy'] = pd.to_numeric(df['lifeexpectancy'])

I want to see how many missing observations there are in the variables of interest:

In [11]:
print ('Countries with a Democracy Score: ' + str(df['polityscore'].count()) + 
       ' out of ' + str(len(df)) + ' (' + str(len(df) - df['polityscore'].count()) + ' missing)')
print ('Countries with a Life Expentacy: ' + str(df['lifeexpectancy'].count()) + 
       ' out of ' + str(len(df)) + ' (' + str(len(df) - df['lifeexpectancy'].count()) + ' missing)')

Countries with a Democracy Score: 161 out of 213 (52 missing)
Countries with a Life Expentacy: 191 out of 213 (22 missing)


I need to have a set of data with both variables of interest so I will subset the data frame:

In [13]:
# Get the rows not missing a value
subset = df[np.isfinite(df['polityscore'])]
subset = subset[np.isfinite(subset['lifeexpectancy'])]
print('Number of observations: '+ str(len(subset)) +' (rows)')

Number of observations: 160 (rows)


160 of the 213 records have complete data. Now I will create my new variable for the democracy score categories. I will do this by defining a function and then using that function to create the new variable:

In [14]:
# This function converts the polity score to a category
def convert_polityscore_to_category(score):
    if score == 10:
        return('1 - Full Democracy')
    elif score > 5:
        return('2 - Democracy')
    elif score > 0:
        return ('3 - Open Anocracy')
    elif score > -6:
        return ('4 - Closed Anocracy')
    else:
        return('5 - Autocracy')

# Now we can use the function to create the new variable
subset['democracy'] = subset['polityscore'].apply(convert_polityscore_to_category)

I will also devide the life expectancy into ranges corresponding to low, second, middle, fourth, high  

In [24]:
subset['lifeexpectancy'] = pd.cut(subset['lifeexpectancy'], 5, labels=['Lowest','Second','Middle','Fourth','Highest'])

## Exploratory Data Analysis

### Democracy Score

The first variable of interest it the polityscore variable. This variable is a measure of the level of openess of the country.

In [27]:
print('Countries by Democracy Score (-10=autocracy & 10=full democracy)')
polity_counts = subset['polityscore'].value_counts()
print(polity_counts)

Countries by Democracy Score (-10=autocracy & 10=full democracy)
 10.0    32
 8.0     19
 9.0     15
 7.0     13
-7.0     12
 6.0     10
 5.0      7
-3.0      6
 0.0      6
-4.0      6
-2.0      5
-1.0      4
-9.0      4
 4.0      4
-6.0      3
 2.0      3
 1.0      3
-10.0     2
-8.0      2
-5.0      2
 3.0      2
Name: polityscore, dtype: int64


In [28]:
print('Percent of Countries by Democracy Score')
polity_percents = subset['polityscore'].value_counts(normalize=True)
print(polity_percents)

Percent of Countries by Democracy Score
 10.0    0.20000
 8.0     0.11875
 9.0     0.09375
 7.0     0.08125
-7.0     0.07500
 6.0     0.06250
 5.0     0.04375
-3.0     0.03750
 0.0     0.03750
-4.0     0.03750
-2.0     0.03125
-1.0     0.02500
-9.0     0.02500
 4.0     0.02500
-6.0     0.01875
 2.0     0.01875
 1.0     0.01875
-10.0    0.01250
-8.0     0.01250
-5.0     0.01250
 3.0     0.01250
Name: polityscore, dtype: float64


There are 32 countries that are full democracies (have a polity score of 10). This is roughly 20% of all the data. There are 2 observations that are autocracies. It seems like most of the countries are greater than zero. I would like to see how many so I will compute a quick percentage:

In [29]:
greater_than_zero = subset[subset['polityscore'] > 0]
greater_than_zero_percent = len(greater_than_zero) * 100 / len(subset)
print('Number of countries with a Polity score greater than zero: ' + str(len(greater_than_zero)))
print('Percent of countries with a Polity score greater than zero: ' + str(greater_than_zero_percent) + '%')

Number of countries with a Polity score greater than zero: 108
Percent of countries with a Polity score greater than zero: 67%


108 of the 160 countries in this data set have some degree of openness. That makes up over half of the data (69%).

## Life Expectancy Per Person Quintiles

In [31]:
print('Countries by Life Expectancy Quintiles')
lifeexpectancy_counts = subset.groupby('lifeexpectancy').size()
print(lifeexpectancy_counts)

Countries by Life Expectancy Quintiles
lifeexpectancy
Lowest     24
Second     17
Middle     29
Fourth     54
Highest    36
dtype: int64


In [33]:
print('Percent of Countries by Life Expectancy Quintiles')
lifeexpectancy_percents = lifeexpectancy_counts * 100 / len(subset)
print(lifeexpectancy_percents)

Percent of Countries by Life Expectancy Quintiles
lifeexpectancy
Lowest     15.000
Second     10.625
Middle     18.125
Fourth     33.750
Highest    22.500
dtype: float64


The data is clustered around the fourth and Highest. We see that 90 (roughly 55%) of the observations are in the fourth and Highest quintiles. There are 36 that are in the highest quintile.

## Democracy Categories

In [35]:
print('Countries by Democracy Category')
democracy_counts = subset.groupby('democracy').size()
print(democracy_counts)

Countries by Democracy Category
democracy
1 - Full Democracy     32
2 - Democracy          57
3 - Open Anocracy      19
4 - Closed Anocracy    29
5 - Autocracy          23
dtype: int64


In [36]:
print('Percent of Countries by Democracy Category')
democracy_percents = democracy_counts * 100 / len(subset)
print(democracy_percents)

Percent of Countries by Democracy Category
democracy
1 - Full Democracy     20.000
2 - Democracy          35.625
3 - Open Anocracy      11.875
4 - Closed Anocracy    18.125
5 - Autocracy          14.375
dtype: float64
